# Retrieving recent events and planning your followup

We've now covered all the endpoints provided by `voeventdb`. In this notebook, we'll look at how to request particular list-orderings, and demonstrate how you might use `voeventdb` to plan a night's follow-up observing.

In [ ]:
from __future__ import print_function
import voeventdb.remote as vr
import voeventdb.remote.apiv1 as apiv1
from voeventdb.remote.apiv1 import FilterKeys
from voeventdb.remote.helpers import Synopsis

... (work in progress)

## Wrapping up

That about covers all the major features of ``voeventdb``, as demonstrated using the ``voeventdb.remote`` client-library. 
Comments, questions, bug-reports and other contributions are all welcomed - you can leave a note on the 
[issue tracker](https://github.com/timstaley/voeventdb.remote/issues) or find more contact details at http://4pisky.org/voevents/.